In [1]:
import requests

import json

import pandas as pd

In [51]:
# GET A DICTIONARY OF STATES

endpoint_url = f'https://cdn-api.co-vin.in/api/v2/admin/location/states'

output_type = "application/json"
language = "en_US"

# CHANGE USER_AGENT FOR YOUR BROWSER / DEVICE
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) \
                Version/14.1 Safari/605.1.15"

query = f'{endpoint_url}'

query_headers = {"Accept" : output_type,
                 "Accept-Language" : language,
                 "User-Agent" : user_agent}

response = requests.get(query, 
               headers = query_headers)

print("response code = ", response.status_code,"\n")

if response.status_code == 200:
    print("response ok \n")
    states = response.json()

elif response.status_code == 403:
    print("Cannot connect to server \n")
    print(response.text)

else:
    print("Other error, see response.text \n")
    print(response.text)

states

response code =  200 

response ok 



{'states': [{'state_id': 1, 'state_name': 'Andaman and Nicobar Islands'},
  {'state_id': 2, 'state_name': 'Andhra Pradesh'},
  {'state_id': 3, 'state_name': 'Arunachal Pradesh'},
  {'state_id': 4, 'state_name': 'Assam'},
  {'state_id': 5, 'state_name': 'Bihar'},
  {'state_id': 6, 'state_name': 'Chandigarh'},
  {'state_id': 7, 'state_name': 'Chhattisgarh'},
  {'state_id': 8, 'state_name': 'Dadra and Nagar Haveli'},
  {'state_id': 37, 'state_name': 'Daman and Diu'},
  {'state_id': 9, 'state_name': 'Delhi'},
  {'state_id': 10, 'state_name': 'Goa'},
  {'state_id': 11, 'state_name': 'Gujarat'},
  {'state_id': 12, 'state_name': 'Haryana'},
  {'state_id': 13, 'state_name': 'Himachal Pradesh'},
  {'state_id': 14, 'state_name': 'Jammu and Kashmir'},
  {'state_id': 15, 'state_name': 'Jharkhand'},
  {'state_id': 16, 'state_name': 'Karnataka'},
  {'state_id': 17, 'state_name': 'Kerala'},
  {'state_id': 18, 'state_name': 'Ladakh'},
  {'state_id': 19, 'state_name': 'Lakshadweep'},
  {'state_id': 20,

In [52]:
# GET A DICTIONARY OF DISTRICTS IN THE STATE

state_id = 9                 # CHANGE STATE_ID TO CHOOSE YOUR STATE OF INTEREST

endpoint_url_dist = f'https://cdn-api.co-vin.in/api/v2/admin/location/districts/{state_id}'

query = f'{endpoint_url_dist}'

response = requests.get(query, 
               headers = query_headers)

print("response code = ", response.status_code,"\n")

if response.status_code == 200:
    print("response ok \n")
    state_districts = response.json()

elif response.status_code == 403:
    print("Cannot connect to server \n")
    print(response.text)

else:
    print("Other error, see response.text \n")
    print(response.text)
    

state_districts


response code =  200 

response ok 



{'districts': [{'district_id': 141, 'district_name': 'Central Delhi'},
  {'district_id': 145, 'district_name': 'East Delhi'},
  {'district_id': 140, 'district_name': 'New Delhi'},
  {'district_id': 146, 'district_name': 'North Delhi'},
  {'district_id': 147, 'district_name': 'North East Delhi'},
  {'district_id': 143, 'district_name': 'North West Delhi'},
  {'district_id': 148, 'district_name': 'Shahdara'},
  {'district_id': 149, 'district_name': 'South Delhi'},
  {'district_id': 144, 'district_name': 'South East Delhi'},
  {'district_id': 150, 'district_name': 'South West Delhi'},
  {'district_id': 142, 'district_name': 'West Delhi'}],
 'ttl': 24}

In [53]:
def appointments(districts, appointment_dates):
    """The function takes in a list of districts and appointment dates in 'dd-mm-yyyy' format
        and returns a dataframe with all available appointments for the specified dates in those districts"""
    
    endpoint_url_app = f'https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/findByDistrict'
    
    district_appointments_df = pd.DataFrame(columns = ['date', 'name', 'address', 'district_name', 'pincode',
                                                       'fee', 'available_capacity', 'vaccine', 'min_age_limit'])
    for district in districts:
        for app_date in appointment_dates:
            query = f'{endpoint_url_app}?district_id={district}&date={app_date}'
            
            query_app = f'{query}'
            
            response = requests.get(query_app, 
               headers = query_headers)
            
#             print(response.status_code)

            if response.status_code != 200:
                print(response.text)

            else:
                district_apps = response.json()
            
            if (response.status_code == 200 and bool(district_apps['sessions']) == True):
                district_apps_df = pd.DataFrame.from_dict(district_apps)
    
                district_apps_df_2 = district_apps_df['sessions'].apply(pd.Series)
    
                dist_apps_df_3 = district_apps_df_2[(district_apps_df_2['available_capacity'] > 0) 
                                        & (district_apps_df_2['min_age_limit'] == 18) # CHANGE AGE LIMIT HERE
                                       ]
    
                dist_apps_df_4 = dist_apps_df_3[['date', 'name', 'address', 'district_name', 'pincode', 'fee',
                                 'available_capacity', 'vaccine','min_age_limit']]
                
#                 print('df_4 created with values for {} district and {} date'.format(district, app_date))

            else:
    
                dist_apps_df_4 = pd.DataFrame(columns = ['date', 'name', 'address', 'district_name', 'pincode', 
                                                         'fee', 'available_capacity', 'vaccine', 'min_age_limit'])
        
#                 print('Blank df_4 created for {} district and {} date'.format(district, app_date))
        
            district_appointments_df.append(dist_apps_df_4)
        
    if len(district_appointments_df.index) == 0:
        print('No Appointments Available for Selected District and Dates.')
        
    else:
        return district_appointments_df

In [59]:
# RUN THIS IF YOU WANT TO CHECK APPOINTMENTS IN ALL DISTRICT OF YOUR STATE
# OTHERWISE, ENTER DISTRICT IDs IN A LIST AND PASS THAT LIST IN THE FUNCTION 
disrtict_ids = {}
district_names = {}
i = 0

for x in state_districts['districts']:
    disrtict_ids[i] = x['district_id']
    district_names[i] = x['district_name']
    i += 1

district_id_list = list(disrtict_ids.values())
district_name_list = list(district_names.values())

In [55]:
# MANUALLY ENTER DISTRICT AND DATES OF INTEREST

# district_list = [141]
app_dates = ['09-05-2021', '10-05-2021', '11-05-2021', '12-05-2021']      # date in 'DD-MM-YYYY' format

In [60]:
# CALLING FUNCTION appointments FOR SELECTED DISTRICT AND DATES
appointments(district_id_list, app_dates)
print('Selected Districts : {}'.format(district_name_list))
print('Selected Dates : {}'.format(app_dates))

No Appointments Available for Selected District and Dates.
Selected Districts : ['Central Delhi', 'East Delhi', 'New Delhi', 'North Delhi', 'North East Delhi', 'North West Delhi', 'Shahdara', 'South Delhi', 'South East Delhi', 'South West Delhi', 'West Delhi']
Selected Dates : ['09-05-2021', '10-05-2021', '11-05-2021', '12-05-2021']
